# Analysis for Day 5: Data Verification & Formulation

## Task 1: Verify Average Profitability Calculation

We attempted to recreate the `average_profitability` in `segments.csv` using the formula:
$$ \text{AvgProfit}(Segment) = \text{Mean}(\text{Asset Quarterly profitability}) + \text{Segment Mean} $$

**Findings:**
- The numbers **do not match** exactly.
- Differences range from -31% to +5%.
- **Hypothesis**: The quarterly data contains outliers (e.g., `InvestmentMortgage` has a 6.0 value in 2020Q4). The `segments.csv` averages likely excluded these outliers or used a robust mean calculation. For example, `InvestmentMortgage_Prime` is -7.86%, while the raw calculation including the outlier yields +23%.

## Task 2: Current Portfolio Metrics

Using the data in `segments.csv`:

- **Total Portfolio Exposure**: **4,068,465**
- **Weighted Average Risk Weight**: **57.81%**
- **Constraint Check**: The 50% regulatory constraint is currently **BINDING** (Violated). The portfolio must reduce risk or shift assets to satisfy this.

## Task 3: Data Concerns

1.  **Negative Profitability**: 4 segments have negative average profitability:
    - `InvestmentMortgage_Prime` (-7.86%)
    - `InvestmentMortgage_Standard` (-1.93%)
    - `RRSPLoan_Prime` (-0.38%)
    - `RVLoan_Prime` (-0.94%)
2.  **InvestmentMortgage Anomalies**: This asset class has extreme variance, ranging from -7.86% (Prime) to +121.68% (Subprime). This is likely driven by data errors (e.g. the 600% return in 2020Q4).

## Task 4: Mathematical Formulation (Draft)

**Decision Variables**:
- $x_s$: New exposure amount for segment $s$ (72 variables).

**Objective Function**:
- Maximize Net Profit: 
$$ \max \sum_{s} x_s \cdot \text{AvgProfit}_s - \text{TransactionCost}(x_s, \text{CurrentExposure}_s) $$

**Constraints**:
1.  **Risk Weight**: $\frac{\sum x_s \cdot w_s}{\sum x_s} \le 0.50$
2.  **Growth Limit**: $\sum x_s \le 1.20 \cdot \sum \text{CurrentExposure}_s$
3.  **Asset Limits**: For each asset class $a$: 
    $$ \text{Current}_a \cdot (1 - \text{MaxDecr}_a) \le \sum_{s \in a} x_s \le \text{Current}_a \cdot (1 + \text{MaxIncr}_a) $$
4.  **Non-negativity**: $x_s \ge 0$


In [ ]:
import pandas as pd
import os

# Load Data
segments = pd.read_csv('Day 5/segments.csv')
assets = pd.read_csv('Day 5/assets.csv')

# Task 2 Calculations
total_exp = segments['exposure'].sum()
risk_wgt = (segments['exposure'] * segments['risk_weight']).sum() / total_exp

print(f"Total Exposure: {total_exp:,.2f}")
print(f"Weighted Risk: {risk_wgt:.2%}")
